### Refining quadrilateral and hexahedral element meshes
https://gitlab.onelab.info/gmsh/gmsh/-/issues/1381

https://stackoverflow.com/questions/65726076/plotting-data-on-a-hexagonal-figure

Remove duplicate points:

https://gitlab.onelab.info/gmsh/gmsh/-/issues/983


In [ ]:
import gmsh
import numpy as np

cl = 9
cg = (0, 0)
DIM = 2

In [ ]:
def get_square_points(square):
    _, adjancecies_down = gmsh.model.getAdjacencies(DIM, square)
    _, pt_tags_adj_down = gmsh.model.getAdjacencies(DIM - 1, adjancecies_down[2])
    return pt_tags_adj_down

def add_line_between_squares(square_0_pts, square_1_pts):
    line_0_0 = gmsh.model.occ.addLine(startTag=square_0_pts[0], endTag=square_1_pts[0])
    line_0_1 = gmsh.model.occ.addLine(startTag=square_0_pts[1], endTag=square_1_pts[1])
    return line_0_0, line_0_1

def add_refinement_connectivity(square_0, square_1, squares_n2, squares_n22):
    square_0_pts = get_square_points(square_0)
    square_1_pts = get_square_points(square_1)
    
    line_0_1, line_0_2 = add_line_between_squares(square_0_pts, square_1_pts)

    squares_n2_pts = np.empty((len(squares_n2), 2), dtype=int)
    for i, square in enumerate(squares_n2):
        pt_tags_adj_down = get_square_points(square)
        squares_n2_pts[i] = pt_tags_adj_down

    squares_n22_pts = np.empty((len(squares_n22), 2), dtype=int)
    for i, square in enumerate(squares_n22):
        pt_tags_adj_down = get_square_points(square)
        squares_n22_pts[i] = pt_tags_adj_down


    lines_connectors = np.empty((len(squares_n2), 2), dtype=int)
    for i in range(3):
        line_0_1, line_0_2 = add_line_between_squares(squares_n2_pts[i], squares_n22_pts[i])
        lines_connectors[i] = [line_0_1, line_0_2]
    gmsh.model.occ.synchronize()
    return lines_connectors

def create_square_geom(cg, cl):
    square_0 = gmsh.model.occ.addRectangle(cg[0] - cl/2, cg[1] - cl/2, 0, cl, cl)
    square_1 = gmsh.model.occ.addRectangle(cg[0] - cl/6, cg[1] - cl/6, 0, cl/3, cl/3)


    square_11 = gmsh.model.occ.addRectangle(cg[0] - cl/2, cg[1] - cl/2, 0, cl/3, cl/3)
    square_12 = gmsh.model.occ.addRectangle(cg[0] - cl/6, cg[1] - cl/2, 0, cl/3, cl/3)
    square_13 = gmsh.model.occ.addRectangle(cg[0] + cl/6, cg[1] - cl/2, 0, cl/3, cl/3)
    squares_n2 = [square_11, square_12, square_13]

    square_111 = gmsh.model.occ.addRectangle(cg[0] - cl/2, cg[1] - cl/2, 0, cl/9, cl/9)
    square_112 = gmsh.model.occ.addRectangle(cg[0] - 3.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)
    square_113 = gmsh.model.occ.addRectangle(cg[0] - 2.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)

    square_211 = gmsh.model.occ.addRectangle(cg[0] - cl/6, cg[1] - cl/2, 0, cl/9, cl/9)
    square_212 = gmsh.model.occ.addRectangle(cg[0] - 0.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)
    square_213 = gmsh.model.occ.addRectangle(cg[0] + 0.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)

    square_311 = gmsh.model.occ.addRectangle(cg[0] + cl/6, cg[1] - cl/2, 0, cl/9, cl/9)
    square_312 = gmsh.model.occ.addRectangle(cg[0] + 2.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)
    square_313 = gmsh.model.occ.addRectangle(cg[0] + 3.5 * cl/9, cg[1] - cl/2, 0, cl/9, cl/9)

    square_122 = gmsh.model.occ.addRectangle(cg[0] - 3.5 * cl/9, cg[1] - 3.5 * cl/9, 0, cl/9, cl/9)
    square_222 = gmsh.model.occ.addRectangle(cg[0] - 0.5 * cl/9, cg[1] - 3.5 * cl/9, 0, cl/9, cl/9)
    square_322 = gmsh.model.occ.addRectangle(cg[0] + 2.5 * cl/9, cg[1] - 3.5 * cl/9, 0, cl/9, cl/9)
    squares_n22 = [square_122, square_222, square_322]
    return square_0, square_1, squares_n2, squares_n22

def remove_square_curves():
    for curve in gmsh.model.getEntities(1):
        gmsh.model.occ.remove([curve])
    gmsh.model.occ.synchronize()

def remove_square_surfaces():
    for plane in gmsh.model.getEntities(2):
        gmsh.model.occ.remove([plane])
    gmsh.model.occ.synchronize()

In [ ]:
# Create a mesh of a square
gmsh.initialize()

# create base geometry
square_0, square_1, squares_n2, squares_n22 = create_square_geom(cg, cl)
gmsh.model.occ.synchronize()
# add refinement connectivity to decrease the mesh size
lines_connectors = add_refinement_connectivity(square_0, square_1, squares_n2, squares_n22)
remove_square_surfaces()
remove_square_curves()

# for line in gmsh.model.getEntities(1):
#     gmsh.model.mesh.setTransfiniteCurve(line[1], 1)
# for surf in gmsh.model.getEntities(2):
#     gmsh.model.mesh.setTransfiniteSurface(surf[1])
# gmsh.model.mesh.setRecombine(2, 1)
# gmsh.model.mesh.generate(2)
gmsh.write("../../../99_testing_prototyping/square.geo_unrolled")
gmsh.fltk.run()
gmsh.finalize()

# check square.geo ;)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
cl = 3
cg = (0, 0)

In [ ]:
# * SINGLE CENTERED SQUARE FUNCTION
def create_square(cg, cl):
    return np.array([[cg[0] - cl/2, cg[1] - cl/2],
                     [cg[0] - cl/2, cg[1] + cl/2],
                     [cg[0] + cl/2, cg[1] + cl/2],
                     [cg[0] + cl/2, cg[1] - cl/2]])

square_center = create_square(cg, cl)
square_bottom = create_square((cg[0], cg[1] - cl), cl)
square_n1 = create_square((cg[0], cg[1] - 4*cl/3), cl/3)
square_n1_bottom = create_square((cg[0], cg[1] - 5*cl/3), cl/3)
square_n11 = create_square((cg[0], cg[1] - 16*cl/9), cl/9)
square_n11_bottom = create_square((cg[0], cg[1] - 17*cl/9), cl/9)

In [11]:
# * MULTIPLE INLINE SQUARES FUNCTION

def sort_trab_ccw(squares_unique_coords):
    squares_center = np.mean(squares_unique_coords, axis=0)
    squares_ccw_idx = np.argsort(np.arctan2(squares_unique_coords[:, 1] - squares_center[1], squares_unique_coords[:, 0] - squares_center[0]))
    return squares_unique_coords[squares_ccw_idx]

def gmsh_add_points(squares_ccw_sort):
    points = np.empty(np.shape(squares_ccw_sort))
    for i, point in enumerate(squares_ccw_sort):
        point_tags = gmsh.model.occ.addPoint(point[0], point[1], 0)
        points[i] = point_tags
    return points
    
def gmsh_create_lines(points):
    print(points)
    lines = []
    for i in range(len(points) - 1):
        print(f'point1 = {points[i]}, point2 = {points[i+1]}')
        # line_tags = gmsh.model.occ.addLine(points[i], points[i+1])
        # lines.append(line_tags)
    return lines

def create_squares_m(cg, cl, n_squares, _dir):
    squares = []
    if _dir == 'vertical':
        for i in range(n_squares):
            center_x = cg[0] + (i - (n_squares - 1)/2) * cl
            square = np.array([[center_x - cl/2, cg[1] - cl/2],
                               [center_x - cl/2, cg[1] + cl/2],
                               [center_x + cl/2, cg[1] + cl/2],
                               [center_x + cl/2, cg[1] - cl/2]])
            squares.append(square)
    
    elif _dir == 'horizontal':
        for i in range(n_squares):
            center_y = cg[1] + (i - (n_squares - 1)/2) * cl
            square = np.array([[cg[0] - cl/2, center_y - cl/2],
                               [cg[0] + cl/2, center_y - cl/2],
                               [cg[0] + cl/2, center_y + cl/2],
                               [cg[0] - cl/2, center_y + cl/2]])
            squares.append(square)
    squares_reshaped = np.array(squares).reshape(-1, 2)
    squares_unique_coords = np.unique(squares_reshaped, axis=0)
    
    # sort points in ccw order
    squares_ccw_sort = sort_trab_ccw(squares_unique_coords)
    point_tags = gmsh_add_points(squares_ccw_sort)
    point_tags = np.array(point_tags, dtype=int)
    point_tags_4 = point_tags.reshape(-1, 4)
    lines = gmsh_create_lines(point_tags_4)
    gmsh.model.occ.synchronize()
    return squares_unique_coords, point_tags

def create_square_directional(cg, cl_i, cl_j):
    cl_t = cl_i + cl_j
    m = 3
    if cl_i == 0:
        _dir_s = 'vertical'
        square_bottom, square_bottom_tags = create_squares_m((cg[0] - cl_i, cg[1] - cl_j), cl_t, 1, _dir=_dir_s)
        square_n1, square_n1_tags = create_squares_m((cg[0] - cl_i, cg[1] - cl_j), cl_t/m, 2*m+1, _dir=_dir_s)
        square_n11, square_n11_tags = create_squares_m((cg[0] - cl_i, cg[1] - (cl_j + cl_j/m)), cl_t/(3*m), 27, _dir=_dir_s)
        square_n11_bottom, square_n11_bottom_tags = create_squares_m((cg[0] - cl_i, cg[1] - (cl_j + cl_j/m + cl_j/(3*m))), cl_t/(3*m), 27, _dir=_dir_s)
    elif cl_j == 0:
        _dir_s = 'horizontal'
        square_bottom, square_bottom_tags = create_squares_m((cg[0] - cl_i, cg[1] - cl_j), cl_t, 1, _dir=_dir_s)
        square_n1, square_n1_tags = create_squares_m((cg[0] - cl_i, cg[1] - cl_j), cl_t/m, 2*m+1, _dir=_dir_s)
        square_n11, square_n11_tags = create_squares_m((cg[0] - (cl_i + cl_i/m), cg[1] - cl_j), cl_t/(3*m), 27, _dir=_dir_s)
        square_n11_bottom, square_n11_bottom_tags = create_squares_m((cg[0] - (cl_i + cl_i/m + cl_i/(3*m)), cg[1] - cl_j), cl_t/(3*m), 27, _dir=_dir_s)
    else:
        raise ValueError("cl_i and cl_j cannot be both different from 0")
    point_coords = np.concatenate((square_bottom, square_n1, square_n11, square_n11_bottom)).tolist()
    point_tags = np.concatenate((square_bottom_tags, square_n1_tags, square_n11_tags, square_n11_bottom_tags)).tolist()
    return point_coords, point_tags

import gmsh
gmsh.initialize()
square_center, square_center_tags = create_squares_m(cg, cl, 1, _dir='horizontal')
square_inf, square_inf_tags = create_square_directional(cg=cg, cl_i=0, cl_j=cl)
# square_l = create_square_directional(cg=cg, cl_i=cl, cl_j=0)
# square_sup = create_square_directional(cg=cg, cl_i=0, cl_j=-cl)
# square_r = create_square_directional(cg=cg, cl_i=-cl, cl_j=0)
gmsh.model.occ.synchronize()
gmsh.fltk.run()
gmsh.finalize()


[[1 1 2 2]
 [3 3 4 4]]
point1 = [1 1 2 2], point2 = [3 3 4 4]
[[5 5 6 6]
 [7 7 8 8]]
point1 = [5 5 6 6], point2 = [7 7 8 8]
[[ 9  9 10 10]
 [11 11 12 12]
 [13 13 14 14]
 [15 15 16 16]
 [17 17 18 18]
 [19 19 20 20]
 [21 21 22 22]
 [23 23 24 24]]
point1 = [ 9  9 10 10], point2 = [11 11 12 12]
point1 = [11 11 12 12], point2 = [13 13 14 14]
point1 = [13 13 14 14], point2 = [15 15 16 16]
point1 = [15 15 16 16], point2 = [17 17 18 18]
point1 = [17 17 18 18], point2 = [19 19 20 20]
point1 = [19 19 20 20], point2 = [21 21 22 22]
point1 = [21 21 22 22], point2 = [23 23 24 24]
[[ 25  25  26  26]
 [ 27  27  28  28]
 [ 29  29  30  30]
 [ 31  31  32  32]
 [ 33  33  34  34]
 [ 35  35  36  36]
 [ 37  37  38  38]
 [ 39  39  40  40]
 [ 41  41  42  42]
 [ 43  43  44  44]
 [ 45  45  46  46]
 [ 47  47  48  48]
 [ 49  49  50  50]
 [ 51  51  52  52]
 [ 53  53  54  54]
 [ 55  55  56  56]
 [ 57  57  58  58]
 [ 59  59  60  60]
 [ 61  61  62  62]
 [ 63  63  64  64]
 [ 65  65  66  66]
 [ 67  67  68  68]
 [ 69  6

In [ ]:
a = 10
fig = plt.figure(figsize=(a, a))
ax = fig.add_subplot(111)
ax.plot(square_inf[:, 0], square_inf[:, 1], 'o', label='square_inf')
ax.plot(square_r[:, 0], square_r[:, 1], 'o', label='square_r')
ax.plot(square_sup[:, 0], square_sup[:, 1], 'o', label='square_sup')
ax.plot(square_l[:, 0], square_l[:, 1], 'o', label='square_l')
ax.plot(square_center[:, 0], square_center[:, 1], 'o', label='square_center')

plt.legend()
ax.set_aspect('equal')

In [ ]:
# concatenate all the squares
points = np.concatenate((square_inf, square_r, square_sup, square_l, square_center))
points_unique = np.unique(points, axis=0)

# sort points based on radius from center
points_sorted = points_unique[np.argsort(np.linalg.norm(points_unique - cg, axis=1))]
# sort points_sorted based on angle from center
points_angle_sorted = points_sorted[np.argsort(np.arctan2(points_sorted[:, 1] - cg[1], points_sorted[:, 0] - cg[0]))]






In [ ]:
import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import random

x, y = points[:, 0], points[:, 1]
# x,y = np.meshgrid(np.arange(10), np.arange(10))
# x = x.flatten()
# y = y.flatten()
coordinates = np.column_stack([x,y])+0.04 * np.random.rand(len(x), 2)
np.random.shuffle(coordinates)
x=coordinates[:,0]
y=coordinates[:,1]

# perform triangulation
triang=tri.Triangulation(x,y)
f = plt.figure(figsize=(10,10))
ax = plt.axes()
tri.triplot(ax,triang)